In [22]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random 
import cv2
import imutils
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
import keras
from keras.utils import np_utils
from keras.models import Model, Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.neighbors import KNeighborsClassifier
from keras.applications.resnet50 import ResNet50
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pickle
import itertools

np.random.seed(123)

In [2]:
import sys

import tensorflow.keras
import sklearn as sk

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.1.0
Keras Version: 2.2.4-tf

Python 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.1
Scikit-Learn 0.24.1
GPU is available


In [38]:
train_dir = 'dataset/handwritten_character/Train'
val_dir = 'dataset/handwritten_character/Validation'

idg = ImageDataGenerator(rescale=1./255)
                       

# get all the data in the directory train (834036 images), and reshape them
train_generator = idg.flow_from_directory(train_dir, color_mode = 'grayscale',
                                         target_size=(32, 32),
                                         batch_size = 834036,
                                         class_mode ='categorical')

# get all the data in the directory validation (22524 images), and reshape them
valid_generator = idg.flow_from_directory(val_dir, color_mode = 'grayscale',
                                         target_size=(32, 32),
                                         batch_size = 22524,
                                         class_mode ='categorical')

# create the data sets
# train_images, train_labels = next(train_generator)
# val_images, val_labels = next(valid_generator)

Found 834036 images belonging to 39 classes.
Found 22524 images belonging to 39 classes.


In [40]:
# Saving the train images/labels
save_path = "pickle_files/"

# with open(os.path.join(save_path, "train_cat.pkl"), "wb") as f:
#     pickle.dump((train_images, train_labels), f)

In [41]:
# Saving the val images/labels

# with open(os.path.join(save_path, "val_cat.pkl"), "wb") as f:
#     pickle.dump((val_images, val_labels), f)

In [26]:
# Load train images/labels
with open(os.path.join(save_path, "train_cat.pkl"), "rb") as f:
    (train_images, train_labels) = pickle.load(f)

In [27]:
# Load val images/labels
with open(os.path.join(save_path, "val_cat.pkl"), "rb") as f:
    (val_images, val_labels) = pickle.load(f)

In [42]:
# Explore your dataset
print('Train images shape: ', train_images.shape)
print('Val images shape: ', val_images.shape)
print('Train labels shape: ', train_labels.shape)
print('Val labels shape: ', val_labels.shape)

Train images shape:  (834036, 32, 32, 1)
Val images shape:  (22524, 32, 32, 1)
Train labels shape:  (834036, 39)
Val labels shape:  (22524, 39)


In [61]:
train_img_unrow = train_images.reshape(834036, -1)

In [64]:
val_img_unrow = val_images.reshape(22524, -1)

In [65]:
np.shape(train_img_unrow)

(834036, 1024)

In [66]:
np.shape(val_img_unrow)

(22524, 1024)

In [67]:
train_labels_final = train_labels.reshape(834036, -1)

In [68]:
np.shape(train_labels_final)

(834036, 39)

In [69]:
val_labels_final = val_labels.reshape(22524, -1)

In [70]:
np.shape(val_labels_final)

(22524, 39)

In [71]:
kVals = np.arange(1,10,2)

for k in kVals:
    
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(train_img_unrow, train_labels_final)
    
    score = model.score(val_img_unrow, val_labels_final)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))

k=1, accuracy=93.55%
k=3, accuracy=91.48%
k=5, accuracy=90.61%
k=7, accuracy=90.18%
k=9, accuracy=89.82%


In [74]:
model_1 = KNeighborsClassifier(n_neighbors=1)
model_1.fit(train_img_unrow, train_labels_final)

score = model_1.score(val_img_unrow, val_labels_final)

In [76]:
score

0.9355354288758657

In [75]:
# Save the model to disk
save_path = "saved_models/"

with open(os.path.join(save_path, "KNN_model.pkl"), "wb") as f:
    pickle.dump(model_1, f)

In [ ]:
# Load model from file

with open(os.path.join(save_path, "KNN_model.pkl"), "rb") as f:
    pickled_model = pickle.load(f)

In [31]:
# Save model

model_4.save('saved_models/CNN_model_4')

In [32]:
# Load model

model_4 = keras.models.load_model('saved_models/CNN_model_4')

In [ ]:
#predict on the KNN model